<a href="https://colab.research.google.com/github/dai-zz-flora/dai-zz-flora.github.com/blob/master/chat_with_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. 准备工作

In [ ]:
# @title ### 1.1 安装依赖
%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!apt -y install -qq aria2
!pip install -q transformers datasets  sentencepiece cpm_kernels mdtex2html accelerate sentencepiece install faiss-gpu  sentence-transformers  PyPDF2 pdfplumber sacremoses

env: TF_CPP_MIN_LOG_LEVEL=1
87 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 87 not upgraded.
Need to get 1,475 kB of archives.
After this operation, 5,959 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 122518 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.15.0-1ubuntu0.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.15.0-1ubuntu0.2) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.35.0-1build1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.35.0-1build1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.35.0-1build1_amd64.deb ...
Unpacking aria2 (1.35.0-1build1) ...
Setting up libc-are

In [ ]:
# @title ## 1.2. 指定数据集


# @markdown ### 本地路径
data_name = "/content/sample_data/california_housing_train.csv"  # @param ["", "/content/sample_data/california_housing_train.csv", "/content/sample_data/mnist_test.csv"]
# @markdown ### 下载路径，格式为csv后缀
download_url = None  # @param 

data_file_name = ""
if download_url:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{download_url}" -d /content/sample_data -o download_data.csv
  data_file_name = "/content/sample_data/download_data.csv"
else:
  data_file_name = data_name




In [ ]:
# @title ## 1.3 定义工具函数
def extract_text_between_a_and_b(text, a, b):
  start_index = text.rfind(a)
  end_index = text.find(b,start_index+3)
    
  if start_index == -1 or end_index == -1:
    return None
    
  return text[start_index + len(a):end_index]

# II. 用THUDM/chatglm-6b来生成代码

In [ ]:
# @title # 2.1. 生成问题描述输入框
from ipywidgets import widgets
from IPython.display import display

textarea = widgets.Textarea(
    value='新生成一列total，值等于每一行的total_bedrooms列和population列相乘,打印total列',  # 填入默认值，可以为空
    placeholder='问题描述',  # 在文本框中显示的提示文本
    description='问题',  # 描述文本，显示在文本框上方
    disabled=False , # 是否禁用
     layout=widgets.Layout(width='800px', height='300px')
)

 
 
display(widgets.HBox([textarea,]))

In [ ]:
# @title ## 2.2. 生成prompt
PROMPT_TEMPLATE = """
写一段python代码，利用pandas库来解决下面的问题:
读取文件 {0} 到一个叫df的DataFrame变量，对df进行操作，{1}



"""
code_desc = textarea.value
# if using_chinese:
#   inputs = translator_tokenizer(textarea.value, return_tensors="pt", max_length=512, truncation=True)
#   outputs = translator_model.generate(
#       inputs["input_ids"], 
#   )

#   english_code_desc = translator_tokenizer.decode(outputs[0], skip_special_tokens=True)


prompt = PROMPT_TEMPLATE.format(data_file_name,code_desc)
print(prompt)


写一段python代码，利用pandas库来解决下面的问题:
读取文件 /content/sample_data/california_housing_train.csv 到一个叫df的DataFrame变量，对df进行操作，新生成一列total，值等于每一行的total_bedrooms列和population列相乘,打印total列






In [ ]:
# @title ## 2.3. 初始化模型，为了节省内存使用chatglm-6b-int4

from transformers import AutoTokenizer, AutoModel
checkpoint = "THUDM/chatglm-6b-int4"
code_tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
code_model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).half().cuda()


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b-int4:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b-int4:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b-int4:
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b-int4:
- modeling_chatglm.py
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.c
Compiling gcc -O3 -fPIC -std=c99 /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.c -shared -o /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.so
Using quantization cache
Applying quantization to glm layers


In [ ]:
# @title ## 2.4. 生成回答，从回答中截取代码

code_model = code_model.eval()
response, history = code_model.chat(code_tokenizer, prompt, history=[])
# 
result_code = extract_text_between_a_and_b(response,"```python","```")
print("---- 这是原回答")
print(response)
print("\n\n--- 这是截取的代码")
print(result_code)

---- 这是原回答
以下是使用pandas库来解决上述问题的代码：

```python
import pandas as pd

# 读取文件
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

# 计算总房价
df['total'] = df['total_bedrooms'] * df['population']

# 打印总房价列
print(df.head())
```

上述代码首先读取了文件 `/content/sample_data/california_housing_train.csv`，将其转换为一个DataFrame `df`。然后，它计算了每一行的 `total_bedrooms` 和 `population` 相乘得到的总和 `total`，并将结果保存到新的列 `total` 中，最后打印 `total` 列的索引和值。


--- 这是截取的代码

import pandas as pd

# 读取文件
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

# 计算总房价
df['total'] = df['total_bedrooms'] * df['population']

# 打印总房价列
print(df.head())



In [ ]:
# @title ## 2.4. 执行代码，并拿到局部变量的
exec(result_code,locals())
df = locals()["df"]


   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -114.31     34.19                15.0       5612.0          1283.0   
1    -114.47     34.40                19.0       7650.0          1901.0   
2    -114.56     33.69                17.0        720.0           174.0   
3    -114.57     33.64                14.0       1501.0           337.0   
4    -114.57     33.57                20.0       1454.0           326.0   

   population  households  median_income  median_house_value      total  
0      1015.0       472.0         1.4936             66900.0  1302245.0  
1      1129.0       463.0         1.8200             80100.0  2146229.0  
2       333.0       117.0         1.6509             85700.0    57942.0  
3       515.0       226.0         3.1917             73400.0   173555.0  
4       624.0       262.0         1.9250             65500.0   203424.0  


In [ ]:
# @title ## 2.5. 清理内存,如果有必要的话
del